In [2]:
# Imports
import pandas as pd
import re
import numpy as np

In [28]:
# Load data and output data stats
file_path = "./mmALL_073120_csv.csv"
# Load CSV file into a pandas DataFrame
def load_csv(file_path):
    df = pd.read_csv(file_path)
    return df

# Perform basic EDA on the DataFrame
def perform_eda(df):
    # Display the number of rows and columns in the DataFrame
    print("Shape:")
    print(df.shape)

    # Display the column names in the DataFrame
    print("Columns:")
    print(df.columns)

    # Check for missing values in the DataFrame
    print("Missing Values:")
    print(df.isnull().sum())

    # Check the data types of the columns in the DataFrame
    print("Data Types:")
    print(df.dtypes)

    # Get list of all countries
    print("Countries")
    print(df["country"].unique())
    print(len(df["country"].unique()))

    # Perform additional EDA tasks as needed

# Example usage
df = load_csv(file_path)
perform_eda(df)

Shape:
(17145, 31)
Columns:
Index(['id', 'country', 'ccode', 'year', 'region', 'protest', 'protestnumber',
       'startday', 'startmonth', 'startyear', 'endday', 'endmonth', 'endyear',
       'protesterviolence', 'location', 'participants_category',
       'participants', 'protesteridentity', 'protesterdemand1',
       'protesterdemand2', 'protesterdemand3', 'protesterdemand4',
       'stateresponse1', 'stateresponse2', 'stateresponse3', 'stateresponse4',
       'stateresponse5', 'stateresponse6', 'stateresponse7', 'sources',
       'notes'],
      dtype='object')
Missing Values:
id                           0
country                      0
ccode                        0
year                         0
region                       0
protest                      0
protestnumber                0
startday                  1906
startmonth                1906
startyear                 1906
endday                    1906
endmonth                  1906
endyear                   1906
protester

In [4]:
# Show unique values for each column
for col in df.columns:
    print(f"For {col} we have {len(df[col].unique())} values {df[col].unique()}")

For id we have 17145 values [ 201990001  201990002  201990003 ... 9102018000 9102019000 9102020000]
For country we have 166 values ['Canada' 'Cuba' 'Haiti' 'Dominican Republic' 'Jamaica' 'Mexico'
 'Guatemala' 'Honduras' 'El Salvador' 'Nicaragua' 'Costa Rica' 'Panama'
 'Colombia' 'Venezuela' 'Guyana' 'Suriname' 'Ecuador' 'Peru' 'Brazil'
 'Bolivia' 'Paraguay' 'Chile' 'Argentina' 'Uruguay' 'United Kingdom'
 'Ireland' 'Netherlands' 'Belgium' 'Luxembourg' 'France' 'Switzerland'
 'Spain' 'Portugal' 'Germany' 'Germany West' 'Germany East' 'Poland'
 'Austria' 'Hungary' 'Czechoslovakia' 'Czech Republic' 'Slovak Republic'
 'Italy' 'Albania' 'Kosovo' 'Serbia' 'Macedonia' 'Croatia' 'Yugoslavia'
 'Bosnia' 'Serbia and Montenegro' 'Montenegro' 'Slovenia' 'Greece'
 'Cyprus' 'Bulgaria' 'Moldova' 'Romania' 'USSR' 'Russia' 'Estonia'
 'Latvia' 'Lithuania' 'Ukraine' 'Belarus' 'Armenia' 'Georgia' 'Azerbaijan'
 'Finland' 'Sweden' 'Norway' 'Denmark' 'Cape Verde' 'Guinea-Bissau'
 'Equatorial Guinea' 'Gambia' '

In [5]:
# Figure out where the missing values in some of the columns are coming from

mask = df[['startday', 'startmonth']].isna().any(axis=1)
missing_rows = df[mask]
print(missing_rows[missing_rows["country"] == "Canada"])
print(df[df["country"] == "Canada"].shape)

           id country  ccode  year         region  protest  protestnumber  \
18  201998000  Canada     20  1998  North America        0              0   
19  201999000  Canada     20  1999  North America        0              0   
24  202001000  Canada     20  2001  North America        0              0   
25  202002000  Canada     20  2002  North America        0              0   
27  202004000  Canada     20  2004  North America        0              0   
38  202013000  Canada     20  2013  North America        0              0   
39  202014000  Canada     20  2014  North America        0              0   

    startday  startmonth  startyear  ...  protesterdemand4  stateresponse1  \
18       NaN         NaN        NaN  ...               NaN             NaN   
19       NaN         NaN        NaN  ...               NaN             NaN   
24       NaN         NaN        NaN  ...               NaN             NaN   
25       NaN         NaN        NaN  ...               NaN             

In [6]:
# 
# Dealing with missing "participants" numbers.
# 

# Checking if NaN protestor_category is from empty rows

mask = df[['participants', "participants_category"]].isna().all(axis=1)
missing_rows = df[mask]
print(missing_rows["protest"].unique())

print(missing_rows[missing_rows["protest"] == 1].to_string(index=False))

# There are only 7 rows where there was a protest but the number of protesters is indeterminate.
# For now, for each case, I'll give very rough guesstimates
 
df.loc[df['id'] == 902005006, "participants"] = 500
df.loc[df['id'] == 921997001, "participants"] = 100
df.loc[df['id'] == 922001001, "participants"] = 1000
df.loc[df['id'] == 922003001, "participants"] = 1000
df.loc[df['id'] == 922006004, "participants"] = 50
df.loc[df['id'] == 931994004, "participants"] = 300
df.loc[df['id'] == 6452013004, "participants"] = 1000


[0 1]
        id     country  ccode  year          region  protest  protestnumber  startday  startmonth  startyear  endday  endmonth  endyear  protesterviolence                                               location participants_category participants            protesteridentity            protesterdemand1 protesterdemand2 protesterdemand3 protesterdemand4  stateresponse1 stateresponse2 stateresponse3 stateresponse4 stateresponse5 stateresponse6 stateresponse7                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [7]:
#
# Standardizing the number of participants based on participants and participants_category
#

# General approach: if "participants_category" exists, use that. 
# Actually, I would rather use participants and process it on my own. "participants_category" is too vague
df["participants"] = df["participants"].astype(str)
df["participants"] = df["participants"].str.lower()
df["participants"] = df["participants"].str.strip()

non_numeric_values = df["participants"][pd.to_numeric(df["participants"], errors='coerce').isna()]
# numeric_values = df["participants"][pd.to_numeric(df["participants"], errors='coerce').notna()]
print(non_numeric_values.unique())
# print(numeric_values.unique())

# Checking for all cases where a certain word appears
# print(df[df["participants"].str.contains('nearly', case=False, na=False)]["participants"].unique())

########################
# Mapping - left contains the phrase that I'm looking for. Right contains the number that I'm going to translate into
replace = {
    "hundreds of thousands": 100000,
    "hundred thousands": 100000,
    "hundreds of thousands to 1.4 million": 1000000,
    "hundreds": 100,
    "hundred": 100,
    "the civil human right front aid 110,000 people; the hong kong police aid about 20,000": 130000,
    "in northern chiang mai province, 200 kamnan and village head; in phetchaburi, about 400 kamnan and village head": 600,
    "dozen": 12,
    'march organiser and legal-sector lawmaker dennis kwok, said  2,000 people joined the protest. police put the turnout at 1,700.': 1850,
    "seven million": 7000000,
    "two million": 2000000,
    "million": 1000000,
    "tens of thousands": 10000,
    "couple thousands": 2000,
    "thousand": 1000,
}

for k,v in replace.items():
    df.loc[df["participants"].str.contains(k, case=False, na=False), 'participants'] = v


########################
# Define the substrings to remove and their replacements
replacements = {
    'more than': '', 
    '+': '', 
    "about": '',
    ">": "",
    "<": "",
    "at least": "",
    "around ": "",
    "1000s": "1000",
    "100s": "100",
    "10000s": "10000",
    "10s": "10",
    "0s": "0", ### What about just this?
    "100000s": "100000",
    "people": "",
    "protesters": "",
    "citizens": "",
    "residents": "",
    "members": "",
    "demonstrators": "",
    "some": "",
    "up to": "",
    "estimated to be": "",
    "estimated": "",
    "campaigners": "",
    "taxis": "",
    "activists": "",
    "almost": "",
    "over": "",
    "local": "",
    "villagers": "",
    "participants": "",
    "supporters": "",
    "parents": "",
    "nearly": "",
    "former estate workers":"",
    "drivers":"",
    "teachers":"",
    "construction workers":"",
    ",": "",
    " to ": "-",
    "as many as": "",
}

# Iterate over the replacements and apply the replacements to the column
for substring, replacement in replacements.items():
    df["participants"] = df["participants"].str.replace(substring, replacement, case=False)


########################
# Dealing with the ranges
# Assuming df is the DataFrame containing the data
# Define a regular expression pattern to match the range
pattern = r'(\d+)\s*-\s*(\d+)'

# Function to calculate the midpoint of a range
def calculate_midpoint(range_str):
    if type(range_str) != str:
        return range_str
    match = re.search(pattern, range_str)
    if match:
        print(range_str)
        start = int(match.group(1))
        end = int(match.group(2))
        midpoint = (start + end) // 2
        return midpoint
    else:
        return range_str

# Apply the calculation to the column
df["participants"] = df["participants"].apply(calculate_midpoint)

########################
# After this processing, things get a little dicey, so we have to handle everything manually.
replace = {
    "between 35,000 and 50,000": 42500,
    "between 100000 & 300000": 200000,
    "between 11000 and 45000": 28000,
    'the organisers said  3000 ; police said the turnout was 830 at its peak': 2000,
    "btw. 20000 & 30000": 25000,
}

for k,v in replace.items():
    df.loc[df["participants"].str.contains(k, case=False, na=False), 'participants'] = v

########################
# Of the remaining, I check if participants_category exists. If so, then use it.
replace = {'50-99': 75, '100-999': 550, '2000-4999': 3500, '1000-1999': 1500, '5000-10000': 7500, '>10000': 10000}
for k, v in replace.items():
    df.loc[(pd.to_numeric(df["participants"], errors='coerce').isna()) & (df['participants_category'] == k), "participants"] = v

########################
# Finally, we make manual guesstimates for those remaining events.
replace = {
    1302005002: 50,
    6601992002: 1000,
    6602002005: 1000,
    7321990014: 1000,
    8501998019: 1000,
    8501999009: 1000,
}

for k, v in replace.items():
    df.loc[df['id'] == k, "participants"] = v

########################
# Conversion into numbers
# Now that participants should all be string representations of numbers, we convert everything
# Starting with NaN
df["participants"] = df["participants"].replace("nan", np.nan)
df["participants"] = pd.to_numeric(df["participants"], errors='coerce')

########################
# Final checks - make sure that all valid protest data has numbers
t = df[(df["protest"] == 1)]
ts = t[pd.to_numeric(t["participants"], errors='coerce').isna()]
print("Final Check")
print(ts["participants"].unique())
# SUCCESS - empty frame.

# Below code was used to filter out those rows that had protest info but didn't have any numbers of protesters available.
# t = df[df["protest"] == 1]
# t = t[t["participants"].notna()]
# ["participants_category"]
# ts = t[pd.to_numeric(t["participants"], errors='coerce').isna()]
# print(ts[ts["participants_category"].isna()][["participants", "sources", "notes"]].to_string())

########################
# Cleanup - convert "nan" back into float(nan)


['1000s' '100s' '10000s' 'nan' '10s' '50+' '250-300' 'hundreds'
 'more than 200' 'dozens' 'a group' 'about 300' 'more than 500' 'busloads'
 'widespread' 'thousands' '100+' '100s-1000s' '100s-1000' '300-1000s'
 '1500+' 'about 2000' 'several hundred' 'tens of thousands' '100s-10000s'
 '1100-10000s' '100000s' '1000+' '>50' '>50000' 'more than 1500 detained'
 '230 families' 'several dozen' '50s' '23000+' '5000+' 'more than 1000'
 '3000+' '15000+' 'hundreds of thousands' '164 arrested' '1000s-10000'
 '5,000,000' '>5000' '1,000,000s' 'several dozen arrests'
 'several thousand' 'more than 400 arrested' '20 arrested' '>100'
 '300 arrested' 'several thousand protesters' 'demonstrators'
 'large demonstrations' 'huge protests' 'hundreds of student protesters'
 'mass protests' 'hundreds of thousands of venezuelans'
 '"rocked by protests"' 'more than seven million people' 'millions'
 'hundreds of youths' 'a general strike that paralysed venezuela'
 'protesters on whom the government security forces

/var/folders/8v/jp6zvvw92yx1gn1xgrd_w07r0000gn/T/ipykernel_37900/3851667223.py:91: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["participants"] = df["participants"].str.replace(substring, replacement, case=False)


In [8]:
#
# Standardize Dates
#

# For start date
df[['startyear', 'startmonth', 'startday']] = df[['startyear', 'startmonth', 'startday']].astype(float)

def create_startdatetime(row):
    startyear = row['startyear']
    startmonth = row['startmonth']
    startday = row['startday']

    result = "empty"    
    if pd.notnull(startyear) and pd.notnull(startmonth) and pd.notnull(startday):
        result = pd.to_datetime(f"{int(startyear)}-{int(startmonth)}-{int(startday)}", errors='coerce')
    elif pd.notnull(startyear) and pd.notnull(startmonth):
        result = pd.to_datetime(f"{int(startyear)}-{int(startmonth)}-1", errors='coerce')
    elif pd.notnull(startyear):
        result = pd.to_datetime(f"{int(startyear)}-1-1", errors='coerce')
    else:
        result = pd.NaT

    # print(f"for row {startyear}-{startmonth}-{startday} got result {result}")
    return result

df['startdatetime'] = df.apply(lambda row: create_startdatetime(row), axis=1)

print(df['startdatetime'].head())

# For end date
df[['endyear', 'endmonth', 'endday']] = df[['endyear', 'endmonth', 'endday']].astype(float)

def create_enddatetime(row):
    endyear = row['endyear']
    endmonth = row['endmonth']
    endday = row['endday']

    result = "empty"    
    if pd.notnull(endyear) and pd.notnull(endmonth) and pd.notnull(endday):
        result = pd.to_datetime(f"{int(endyear)}-{int(endmonth)}-{int(endday)}", errors='coerce')
    elif pd.notnull(endyear) and pd.notnull(endmonth):
        result = pd.to_datetime(f"{int(endyear)}-{int(endmonth)}-1", errors='coerce')
    elif pd.notnull(endyear):
        result = pd.to_datetime(f"{int(endyear)}-1-1", errors='coerce')
    else:
        result = pd.NaT

    # print(f"for row {endyear}-{endmonth}-{endday} got result {result}")
    return result

df['enddatetime'] = df.apply(lambda row: create_enddatetime(row), axis=1)

print(df['enddatetime'].head())

0   1990-01-15
1   1990-06-25
2   1990-07-01
3   1990-07-12
4   1990-08-14
Name: startdatetime, dtype: datetime64[ns]
0   1990-01-15
1   1990-06-25
2   1990-07-01
3   1990-09-06
4   1990-08-15
Name: enddatetime, dtype: datetime64[ns]


In [9]:
# 
# Dealing with . in Protester Demand and State Response
# 

In [10]:
#
# Standardize Protestor Demand
#

# Replacing commas
replacements = {
    ",": " and",
}

# Iterate over the replacements and apply the replacements to the column
for substring, replacement in replacements.items():
    df["protesterdemand1"] = df["protesterdemand1"].str.replace(substring, replacement, case=False)
    df["protesterdemand2"] = df["protesterdemand2"].str.replace(substring, replacement, case=False)
    df["protesterdemand3"] = df["protesterdemand3"].str.replace(substring, replacement, case=False)
    df["protesterdemand4"] = df["protesterdemand4"].str.replace(substring, replacement, case=False)

# We want to combine into a single column as a list and order alphabetically
comb = set()

for column in ["protesterdemand1", "protesterdemand2", "protesterdemand3", "protesterdemand4"]:
    unique_values = df[column].unique()
    comb.update(unique_values)

print(comb)

import pandas as pd
import numpy as np

df['alldemands'] = df.apply(
    lambda row: [value for value in row[['protesterdemand1', 'protesterdemand2', 'protesterdemand3', 'protesterdemand4']] if pd.notnull(value)],
    axis=1
)

# Print the updated DataFrame
print(df["alldemands"])

# End by deleting all protester demands
del df["protesterdemand1"]
del df["protesterdemand2"]
del df["protesterdemand3"]
del df["protesterdemand4"]


{'.', 'labor wage dispute', 'removal of politician', 'land farm issue', 'police brutality', 'political behavior and process', 'price increases and tax policy', 'social restrictions', nan}
0        [political behavior and process, labor wage di...
1                         [political behavior and process]
2                         [political behavior and process]
3                                        [land farm issue]
4                         [political behavior and process]
                               ...                        
17140                     [political behavior and process]
17141                     [political behavior and process]
17142                                                   []
17143                                                  [.]
17144                                                   []
Name: alldemands, Length: 17145, dtype: object


In [11]:
#
# Standardize State Response
#

# Replacing commas
replacements = {
    ",": " and",
}

# Iterate over the replacements and apply the replacements to the column
for substring, replacement in replacements.items():
    df["stateresponse1"] = df["stateresponse1"].str.replace(substring, replacement, case=False)
    df["stateresponse2"] = df["stateresponse2"].str.replace(substring, replacement, case=False)
    df["stateresponse3"] = df["stateresponse3"].str.replace(substring, replacement, case=False)
    df["stateresponse4"] = df["stateresponse4"].str.replace(substring, replacement, case=False)
    df["stateresponse5"] = df["stateresponse5"].str.replace(substring, replacement, case=False)
    df["stateresponse6"] = df["stateresponse6"].str.replace(substring, replacement, case=False)
    df["stateresponse7"] = df["stateresponse7"].str.replace(substring, replacement, case=False)

# We want to combine into a single column as a list and order alphabetically
comb = set()

for column in ["stateresponse1", "stateresponse2", "stateresponse3", "stateresponse4", "stateresponse5", "stateresponse6", "stateresponse7"]:
    unique_values = df[column].unique()
    comb.update(unique_values)

print(comb)

import pandas as pd
import numpy as np

df['allresponses'] = df.apply(
    lambda row: [value for value in row[["stateresponse1", "stateresponse2", "stateresponse3", "stateresponse4", "stateresponse5", "stateresponse6", "stateresponse7"]] if pd.notnull(value)],
    axis=1
)

# Print the updated DataFrame
print(df["allresponses"])

# End by deleting all state responses
del df["stateresponse1"]
del df["stateresponse2"]
del df["stateresponse3"]
del df["stateresponse4"]
del df["stateresponse5"]
del df["stateresponse6"]
del df["stateresponse7"]

{'.', 'shootings', 'arrests', 'crowd dispersal', 'accomodation', 'ignore', 'beatings', 'killings', nan}
0                                        [ignore]
1                                        [ignore]
2                                        [ignore]
3                                  [accomodation]
4        [crowd dispersal, arrests, accomodation]
                           ...                   
17140                           [crowd dispersal]
17141                                    [ignore]
17142                                          []
17143                                         [.]
17144                                          []
Name: allresponses, Length: 17145, dtype: object


In [12]:
#
# Extract keywords
#

# Combine sources+notes into single 
df["sources_notes"] = df["sources"] + " " + df["notes"]

# Use summa to extract keywords from sources+notes and put into new keywords column.
from summa import keywords

sentence = "world briefing americas: guatemala: protesters and police clash the new york times, march 17, 2005 thursday, section a; column 4; foreign desk; pg. 6, 82 words, reuters; eighth day of protests over trade agreement with the u.s."

# Extract key topics using TextRank algorithm
def summa_keywords(sentence):
    if type(sentence) != str:
        return []
    key_topics = keywords.keywords(sentence)
    return key_topics.split("\n")

################################
# Apply summa to every df["sources_notes"] and view all uniques
df['keywords'] = df['sources_notes'].apply(lambda x: summa_keywords(x))

################################
# View unique values
from functools import reduce

# Assuming df is the DataFrame containing the column of lists
unique_values = reduce(lambda x, y: x.union(y), df['keywords'].apply(set))

print(unique_values)

del df["sources_notes"]


{'', 'recommendations', 'farage', 'clinging', 'briefs spain phone workers strike', 'stronghold heavily', 'renewed violence', 'tv bbc monitoring', 'ukrainian forces', 'thousands say japan', 'siege declared', 'gasoline prices', 'owns', 'words orthodox protesters', 'impact', 'speech', 'colony bordering', 'central bank', 'vilnius', 'revealing', 'demonstrators urge release', 'saw violent', 'dombrovskis', 'taiwan people', 'backed', 'square outside', 'officals', 'military declares martial', 'purchase', 'protests demand president', 'belfast police chief', 'new clashes', 'genuine universal', 'eu workers', 'yugoslav army', 'trucker', 'bin', 'faso', 'relaxed working', 'peaceful anti', 'fortnight gathered', 'police clubbed', 'stones', 'tania', 'britain', 'earlier escalated', 'muslim ruler', 'leader says papua', 'mackay', 'seize', 'macedonia media review', 'run news', 'pilgrimage', 'drew spontaneous', 'cybersecurity', 'russian coal', 'roundup bangladesh', 'inverleith', 'mamadrizoyev', 'protest land

In [13]:
#
# Final checks of initial cleaning
#
# Here we do final checks on the dataset before this first version is able to be used in webapps

# Past this point, we're going to augment the dataset with additional data sources.

In [25]:
#
# Augment with country iso information
#

# First lets list all the countries
print(df["country"].unique())

import pycountry
# Lets get their ISO numbers
not_found = []
found = {}

for name in df["country"].unique():
    try:
        country = pycountry.countries.get(name=name)
        found[name] = country.numeric
    except Exception as e:
        not_found.append(name)
        print(f"Error converting {name} to ISO code: {e}")
print(not_found)

# Deal with historical countries
# Actually, former countries will not be included
# temp = []
# for name in not_found:
#     try:
#         country = pycountry.historic_countries.get(name=name)
#         found[name] = country.numeric
#     except Exception as e:
#         temp.append(name)
#         print(f"Error converting {name} to ISO code: {e}")
# not_found = temp
# print(not_found)

import country_converter as coco
# Attempt the not_found with country_converter
temp = []
for name in not_found:
    try:
        standard_names = coco.convert(names=name, to='ISOnumeric')
    except Exception as e:
        temp.append(name)
        print(f"Error with {name}")
print(temp)


################################
# Given the dict of mappings, must add new column with countrycode

df.loc[df["country"].isin(found.keys()), 'countrycode'] = df["country"].map(found)
print(df.head().to_string())

################################
# Remove all those entries with NaN country codes
print(df.shape)
df = df.dropna(subset=['countrycode'])
print(df.shape)

Germany West not found in regex


['Canada' 'Cuba' 'Haiti' 'Dominican Republic' 'Jamaica' 'Mexico'
 'Guatemala' 'Honduras' 'El Salvador' 'Nicaragua' 'Costa Rica' 'Panama'
 'Colombia' 'Venezuela' 'Guyana' 'Suriname' 'Ecuador' 'Peru' 'Brazil'
 'Bolivia' 'Paraguay' 'Chile' 'Argentina' 'Uruguay' 'United Kingdom'
 'Ireland' 'Netherlands' 'Belgium' 'Luxembourg' 'France' 'Switzerland'
 'Spain' 'Portugal' 'Germany' 'Germany West' 'Germany East' 'Poland'
 'Austria' 'Hungary' 'Czechoslovakia' 'Czech Republic' 'Slovak Republic'
 'Italy' 'Albania' 'Kosovo' 'Serbia' 'Macedonia' 'Croatia' 'Yugoslavia'
 'Bosnia' 'Serbia and Montenegro' 'Montenegro' 'Slovenia' 'Greece'
 'Cyprus' 'Bulgaria' 'Moldova' 'Romania' 'USSR' 'Russia' 'Estonia'
 'Latvia' 'Lithuania' 'Ukraine' 'Belarus' 'Armenia' 'Georgia' 'Azerbaijan'
 'Finland' 'Sweden' 'Norway' 'Denmark' 'Cape Verde' 'Guinea-Bissau'
 'Equatorial Guinea' 'Gambia' 'Mali' 'Senegal' 'Benin' 'Mauritania'
 'Niger' 'Ivory Coast' 'Guinea' 'Burkina Faso' 'Liberia' 'Sierra Leone'
 'Ghana' 'Togo' 'Camer

Germany East not found in regex
Yugoslavia not found in regex
Serbia and Montenegro not found in regex
USSR not found in regex


[]
          id country  ccode  year         region  protest  protestnumber  startday  startmonth  startyear  endday  endmonth  endyear  protesterviolence          location participants_category  participants           protesteridentity                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     sources                                                                                                

In [26]:
#
# Saving as JSON
#

df.to_json('output.json', orient='records')
df.to_json('data.json.gz', orient='records', compression='gzip')